# Hackathon

Some utilities

## Import Utils

In [27]:
import keras
from keras.metrics import top_k_categorical_accuracy
import h5py as h5
import numpy as np
import pandas as pd
import tables
import matplotlib.pyplot as plt

PATH_DATA = '/notebooks/data/full.h5'
OUT_PATH = '/notebooks/data/full_all_classes.h5'

In [28]:
batch_size = 32
nb_samples = 1869824
batch_count = int(nb_samples/batch_size) + 1

print(batch_count)

58433


In [29]:
label = h5.File(PATH_DATA)['TOP_LANDCOVER']
images = h5.File(PATH_DATA)['S2']

In [30]:
def get_idxs(h5_path):
    f = h5.File(h5_path)
    return range(len(f['S2']))

def shuffle_idx(sample_idxs):
    return list(np.random.permutation(sample_idxs))

def split_train_val(sample_idxs, proportion):
    n_samples = len(sample_idxs)
    return sample_idxs[:int((1.-proportion)*n_samples)], sample_idxs[int((1.-proportion)*n_samples):]

def get_batch_count(idxs, batch_size):
    batch_count = int(len(idxs)//batch_size)
    remained_samples = len(idxs)%batch_size
    if remained_samples > 0:
        batch_count += 1

    return batch_count

In [32]:
idxs = get_idxs(PATH_DATA)
shuffled_idxs = shuffle_idx(idxs)

In [33]:
import os

if os.path.exists(OUT_PATH):
        os.remove(OUT_PATH)
        
f = h5.File(OUT_PATH, 'w')
top_landcover_submit = f.create_dataset("TOP_LANDCOVER", [0, 1], maxshape=(None, 1))
s2_submit = f.create_dataset("S2", [0, 16,16,4], maxshape=(None, 16,16,4))

for b in range(batch_count):
    
    i = shuffled_idxs[b]
    
    top_landcover = h5.File(PATH_DATA)['TOP_LANDCOVER'][i*batch_size:(i+1)*batch_size -1]
    top_landcover = top_landcover[0:].astype(int)
    images = h5.File(PATH_DATA)['S2'][i*batch_size:(i+1)*batch_size -1, :, :, :]
    
#     top_landcover = h5.File(PATH_DATA)['TOP_LANDCOVER'][i*batch_size:(i+1)*batch_size-1]
#     top_landcover = top_landcover[0:].astype(int)
#     images = h5.File(PATH_DATA)['S2'][i*batch_size:(i+1)*batch_size -1, ::]

    top_landcover_submit.resize(top_landcover_submit.shape[0] + batch_size, axis=0)
    top_landcover_submit[-len(top_landcover):] = top_landcover[:]

    s2_submit.resize(s2_submit.shape[0] + batch_size, axis=0)
    s2_submit[-len(top_landcover):, ::] = images

f.close()
                

TypeError: Can't broadcast (0, 1) -> (64, 1)